In [1]:
import logging
from files.backend_codes import build_workflow, metadata_retrieval_node, sql_node, interpret_node, chart_node, processing_node, analysis_plan_node
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, ToolMessage, AIMessage, HumanMessage

# 基本的なロギングを設定
logging.basicConfig(level=logging.INFO)

/tmp/ipykernel_19228/2791776522.py:2: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from files.backend_codes import build_workflow, metadata_retrieval_node, sql_node, interpret_node, chart_node, processing_node, analysis_plan_node
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you'r

In [5]:
res = metadata_retrieval_node.func("データベース内のテーブルの一覧を取得したい", [])
print(res)


このデータベースには、4つのテーブルがあります。それぞれ、どのようなデータが格納されているか説明します。

1. **users テーブル:** ユーザーに関する情報を管理します。

   * `user_id` (INTEGER): ユーザーを識別するためのIDで、主キーです。各ユーザーはユニークなIDを持ちます。
   * `user_name` (TEXT): ユーザーの名前です。文字列で保存されます。
   * `age` (INTEGER): ユーザーの年齢です。整数で保存されます。
   * `prefecture` (TEXT): ユーザーの居住都道府県です。文字列で保存されます。


2. **products テーブル:** 商品に関する情報を管理します。

   * `product_id` (INTEGER): 商品を識別するためのIDで、主キーです。各商品はユニークなIDを持ちます。
   * `product_name` (TEXT): 商品の名前です。文字列で保存されます。
   * `category` (TEXT): 商品のカテゴリです。例えば、「食品」、「家電」などです。文字列で保存されます。
   * `price` (INTEGER): 商品の価格です。整数で保存されます。


3. **reviews テーブル:** 商品に対するレビュー情報を管理します。

   * `review_id` (INTEGER): レビューを識別するためのIDで、主キーです。各レビューはユニークなIDを持ちます。
   * `product_id` (INTEGER): レビュー対象の商品のIDです。`products`テーブルの`product_id`と関連付けられています。
   * `rating` (INTEGER): レビューの評価です。1から5までの整数で、1が最低評価、5が最高評価を表します。
   * `comment` (TEXT): レビューのコメントです。文字列で保存されます。


4. **sales_data テーブル:** 商品の売上に関する情報を管理します。

   * `id` (INTEGER): レコードを識別するためのIDで、主キーです。各売上データはユニークなIDを持ちます。
   * `category

In [6]:
res = analysis_plan_node.func("ブランドの状況が知りたい", [])
print(res)

INFO:root:analysis_plan_node: RAG情報を読み込み中 'ブランドの状況が知りたい'
INFO:root:analysis_plan_node: 生成AIが考えています・・・ 'ブランドの状況が知りたい'


## 分析計画：ブランド状況把握

**目標:**  提供されたデータに基づき、ブランドの状況を分析する。現状では「ブランド」に関する情報が直接テーブルに含まれていないため、間接的に状況を推測する必要がある。  そのため、以下の仮定に基づき分析を進める。

**仮定:**

* 「ブランド」は商品カテゴリ(`products`テーブルの`category`カラム)と関連付けられる。
* ブランドの状況は、売上高、顧客評価、人気商品などから推測できる。


**分析ステップ:**

1. **データ準備:**
    * 各テーブルのデータの整合性チェック（欠損値、異常値の確認）。
    * 必要に応じてデータクレンジング（例: `date`カラムのデータ型変換）。

2. **ブランド別売上分析:**
    * `sales_data`テーブルと`products`テーブルを`category`カラムをキーに結合。
    * 各`category`（ブランドと仮定）ごとの売上高の集計。
    * 売上高の推移（時間軸）を分析するために、`date`カラムを用いて期間ごとの売上高を計算。
    * 売上高ランキングを作成し、トップブランドを特定。
    * グラフ化して視覚的に表現する（棒グラフ、折れ線グラフなど）。

3. **ブランド別顧客評価分析:**
    * `reviews`テーブルと`products`テーブルを`product_id`カラムをキーに結合。
    * `products`テーブルの`category`カラムを用いて、ブランドごとの平均評価を計算。
    * 評価分布（ヒストグラム）を作成し、各ブランドの顧客満足度を比較。

4. **ブランド別人気商品分析:**
    * `sales_data`テーブルを`category`カラムでグループ化し、各ブランドで最も売れた商品を特定。
    * `reviews`テーブルと結合し、人気商品の評価も確認。

5. **総合的なブランド状況評価:**
    * 上記の分析結果を総合的に判断し、各ブランドの強みと弱みを特定。
    * 売上高、顧客評価、人気商品などを考慮した総合的な評価指標を作成（例：加重平均）。

6. **結果の可視化と報告:**
    * 分析結

In [5]:
#SQL取得テスト
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], lambda x, y: x + y]
    next: str  # 次に実行すべきノード名
    df_history: List[dict] 

# テスト用の状態を作成
state = AgentState(
    messages=[
        AIMessage(content="", tool_calls=[{
            # 必須キー 'name' を追加
            # スーパーバイザーがディスパッチ指示を出すツール名
            "name": "DispatchDecision",  
            
            # 必須キー 'args' を追加
            # ダミーの引数でOK
            "args": {                  
                "next_agent": "sql_node",
                "task_description": "ユーザーごとの売上を取得して"
            },
            
            # idはToolMessageと紐付けるために必要
            "id": "call_123"
        }]),
        # このToolMessageは、スーパーバイザーが生成した具体的な指示
        ToolMessage(content="カテゴリごとの売上を取得して", tool_call_id="call_123")
    ],
    df_history=[]
)
res = sql_node(state)
print(res)

INFO:root:sql_node: 要件を処理中: 'カテゴリごとの売上を取得して'
INFO:root:sql_node: 生成AIが考えています・・・ 'カテゴリごとの売上を取得して'


{'messages': [ToolMessage(content='{"status": "success", "executed_sql": "SELECT category, SUM(price) FROM products GROUP BY category", "dataframe_as_json": "[{\\"category\\":\\"\\\\u98df\\\\u54c1\\",\\"SUM(price)\\":450},{\\"category\\":\\"\\\\u98f2\\\\u6599\\",\\"SUM(price)\\":220}]"}', tool_call_id='call_123')], 'df_history': [{'カテゴリごとの売上を取得して': [{'category': '食品', 'SUM(price)': 450}, {'category': '飲料', 'SUM(price)': 220}]}]}


In [2]:
#解釈テスト
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], lambda x, y: x + y]
    next: str  # 次に実行すべきノード名
    df_history: List[dict] 

# --- 1. テストの準備: モックデータを作成 ---

# 1つ目のデータ（月別売上）
sales_data = [
    {"月": "4月", "売上": 100},
    {"月": "5月", "売上": 120},
    {"月": "6月", "売上": 150}
]

# 2つ目のデータ（カテゴリ別顧客数）
customer_data = [
    {"カテゴリ": "A", "顧客数": 50},
    {"カテゴリ": "B", "顧客数": 80}
]

# --- 2. interpret_nodeに渡すテスト用の状態(state)を作成 ---

test_state_for_interpret = AgentState(
    # messagesには、interpret_nodeが呼び出される直前までの会話履歴を模倣して設定
    messages=[
        # 過去のやり取り (省略しても良いが、文脈理解のテストとして含めるとより良い)
        HumanMessage(content="月別売上とカテゴリ別の顧客数を分析したい"),
        AIMessage(content="", tool_calls=[{"name":"DispatchDecision", "args":{}, "id":"call_sql_1"}]),
        ToolMessage(content="月別売上を取得します", tool_call_id="call_sql_1"),
        ToolMessage(content='{"status":"success", "dataframe_as_json": "[...]"}"', tool_call_id="call_sql_1"), # sql_nodeの成功報告
        
        AIMessage(content="", tool_calls=[{"name":"DispatchDecision", "args":{}, "id":"call_sql_2"}]),
        ToolMessage(content="カテゴリ別の顧客数を取得します", tool_call_id="call_sql_2"),
        ToolMessage(content='{"status":"success", "dataframe_as_json": "[...]"}"', tool_call_id="call_sql_2"), # sql_nodeの成功報告

        # interpret_nodeを呼び出す直前の、スーパーバイザーの判断
        AIMessage(
            content="", 
            tool_calls=[{
                "name": "DispatchDecision",
                "args": {                  
                    "next_agent": "interpret_node",
                    "task_description": "これまでの分析結果を総合的に解釈し、ビジネス上の示唆をまとめてください。"
                },
                "id": "call_interpret_1"
            }]
        ),
        # interpret_nodeへの具体的な指示メッセージ
        ToolMessage(
            content="これまでの分析結果を総合的に解釈し、ビジネス上の示唆をまとめてください。",
            tool_call_id="call_interpret_1"
        )
    ],
    
    # df_historyに、過去のノードが生成したデータを設定
    df_history=[
        {"月別売上の取得": sales_data},
        {"カテゴリ別顧客数の取得": customer_data}
    ]
)

res = interpret_node(test_state_for_interpret )
print(res)

INFO:root:interpret_node: df_historyの読み込み開始・・・ 'これまでの分析結果を総合的に解釈し、ビジネス上の示唆をまとめてください。'
INFO:root:interpret_node: 生成AIが考えています・・・ 'これまでの分析結果を総合的に解釈し、ビジネス上の示唆をまとめてください。'


{'messages': [ToolMessage(content='売上は4月から6月にかけて増加傾向にあり、成長が見込めます。顧客数はカテゴリBがAより多く、カテゴリBへの注力や、カテゴリAの顧客獲得戦略の検討がビジネス上の示唆となります。', tool_call_id='call_interpret_1')]}


In [2]:
#グラフ化テスト
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], lambda x, y: x + y]
    next: str  # 次に実行すべきノード名
    df_history: List[dict] 

# 1つ目のデータ（月別売上）
sales_data = [
    {"月": "4月", "売上": 100},
    {"月": "5月", "売上": 120},
    {"月": "6月", "売上": 150}
]

# 2つ目のデータ（カテゴリ別顧客数）
customer_data = [
    {"カテゴリ": "A", "顧客数": 50},
    {"カテゴリ": "B", "顧客数": 80}
]

test_state_for_chart = AgentState(
    # messagesには、interpret_nodeが呼び出される直前までの会話履歴を模倣して設定
    messages=[
        # 過去のやり取り (省略しても良いが、文脈理解のテストとして含めるとより良い)
        HumanMessage(content="月別売上とカテゴリ別の顧客数を分析したい"),
        AIMessage(content="", tool_calls=[{"name":"DispatchDecision", "args":{}, "id":"call_sql_1"}]),
        ToolMessage(content="月別売上を取得します", tool_call_id="call_sql_1"),
        ToolMessage(content='{"status":"success", "dataframe_as_json": "[...]"}"', tool_call_id="call_sql_1"), # sql_nodeの成功報告
        
        AIMessage(content="", tool_calls=[{"name":"DispatchDecision", "args":{}, "id":"call_sql_2"}]),
        ToolMessage(content="カテゴリ別の顧客数を取得します", tool_call_id="call_sql_2"),
        ToolMessage(content='{"status":"success", "dataframe_as_json": "[...]"}"', tool_call_id="call_sql_2"), # sql_nodeの成功報告

        # interpret_nodeを呼び出す直前の、スーパーバイザーの判断
        AIMessage(
            content="", 
            tool_calls=[{
                "name": "DispatchDecision",
                "args": {                  
                    "next_agent": "interpret_node",
                    "task_description": "これまでの分析結果を総合的に解釈し、ビジネス上の示唆をまとめてください。"
                },
                "id": "call_interpret_1"
            }]
        ),
        # interpret_nodeへの具体的な指示メッセージ
        ToolMessage(
            content="月別売上のグラフ化をしてください。",
            tool_call_id="call_interpret_1"
        )
    ],
    
    # df_historyに、過去のノードが生成したデータを設定
    df_history=[
        {"月別売上の取得": sales_data},
        {"カテゴリ別顧客数の取得": customer_data}
    ]
)
res = chart_node(test_state_for_chart )
# print(res)for chart in state["chart_history"]:
#     for task, plotly_json in chart.items():
#         print(f"--- {task} ---")
#         fig = pio.from_json(plotly_json)
#         fig.show()


INFO:root:chart_node: df_historyの読み込み開始・・・ '月別売上のグラフ化をしてください。'
INFO:root:chart_node: 生成AIが考えています・・・ '月別売上のグラフ化をしてください。'




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import plotly.express as px\ndf = df0\nfig = px.line(df, x='月', y='売上', title='月別売上')\nprint(fig.to_json())"}`


{"data":[{"hovertemplate":"月=%{x}\u003cbr\u003e売上=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":["4月","5月","6月"],"xaxis":"x","y":{"dtype":"i2","bdata":"ZAB4AJYA"},"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth"

INFO:root:chart_node: Agent response: {'input': '\n    あなたはPythonプログラミングとデータ可視化の専門家です。\n    現在のタスクと文脈を踏まえて、最適なデータを選択し、最適なグラフを作成してください。\n\n    【現在のタスク】\n    月別売上のグラフ化をしてください。\n    \n    【ユーザーの全体的な質問の文脈】\n    ai: \ntool: カテゴリ別の顧客数を取得します\ntool: {"status":"success", "dataframe_as_json": "[...]"}"\nai: \ntool: 月別売上のグラフ化をしてください。\n\n    【利用可能なデータの内容】\n    \n                    # df0\n                    ## 内容:【月別売上の取得】に関するデータ    \n                    ## dfの列情報: [\'月\', \'売上\']\n                    ## dfの最初の5行:\n 月  売上\n4月 100\n5月 120\n6月 150\n                    \n\n                    # df1\n                    ## 内容:【カテゴリ別顧客数の取得】に関するデータ    \n                    ## dfの列情報: [\'カテゴリ\', \'顧客数\']\n                    ## dfの最初の5行:\nカテゴリ  顧客数\n   A   50\n   B   80\n                    \n\n    最適なインタラクティブグラフを `plotly.express` (例: `px`) を使用して生成してください。\n    生成したFigureオブジェクトを `fig` という変数に格納し、その後 `fig.to_json()` を呼び出してJSON文字列に変換し、そのJSON文字列を `print` してください。\n\n    実行例:\n    import plotly.express as px\n

```json
{"data": [{"hovertemplate": "月=%{x}<br>売上=%{y}<extra></extra>", "legendgroup": "", "line": {"color": "#636efa", "dash": "solid"}, "marker": {"symbol": "circle"}, "mode": "lines", "name": "", "orientation": "v", "showlegend": false, "type": "scatter", "x": ["4月", "5月", "6月"], "xaxis": "x", "y": {"bdata": "ZAB4AJYA", "dtype": "i2"}, "yaxis": "y"}], "layout": {"legend": {"tracegroupgap": 0}, "template": {"data": {"bar": [{"error_x": {"color": "#2a3f5f"}, "error_y": {"color": "#2a3f5f"}, "marker": {"line": {"color": "#E5ECF6", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "bar"}], "barpolar": [{"marker": {"line": {"color": "#E5ECF6", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "barpolar"}], "carpet": [{"aaxis": {"endlinecolor": "#2a3f5f", "gridcolor": "white", "linecolor": "white", "minorgridcolor": "white", "startlinecolor": "#2a3f5f"}, "baxis": {"endlinecolor": "#2a3f5f", "gridcolor": "white",

KeyError: 'tool_calls'

In [6]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], lambda x, y: x + y]
    next: str  # 次に実行すべきノード名
    df_history: List[dict] 

# 1つ目のデータ（月別売上）
sales_data = [
    {"月": "4月", "売上": 100},
    {"月": "5月", "売上": 120},
    {"月": "6月", "売上": 150}
]

# 2つ目のデータ（カテゴリ別顧客数）
customer_data = [
    {"カテゴリ": "A", "顧客数": 50},
    {"カテゴリ": "B", "顧客数": 80}
]

test_state_for_process = AgentState(
    # messagesには、interpret_nodeが呼び出される直前までの会話履歴を模倣して設定
    messages=[
        # 過去のやり取り (省略しても良いが、文脈理解のテストとして含めるとより良い)
        HumanMessage(content="月別売上とカテゴリ別の顧客数を分析したい"),
        AIMessage(content="", tool_calls=[{"name":"DispatchDecision", "args":{}, "id":"call_sql_1"}]),
        ToolMessage(content="月別売上を取得します", tool_call_id="call_sql_1"),
        ToolMessage(content='{"status":"success", "dataframe_as_json": "[...]"}"', tool_call_id="call_sql_1"), # sql_nodeの成功報告
        
        AIMessage(content="", tool_calls=[{"name":"DispatchDecision", "args":{}, "id":"call_sql_2"}]),
        ToolMessage(content="カテゴリ別の顧客数を取得します", tool_call_id="call_sql_2"),
        ToolMessage(content='{"status":"success", "dataframe_as_json": "[...]"}"', tool_call_id="call_sql_2"), # sql_nodeの成功報告

        # interpret_nodeを呼び出す直前の、スーパーバイザーの判断
        AIMessage(
            content="", 
            tool_calls=[{
                "name": "DispatchDecision",
                "args": {                  
                    "next_agent": "interpret_node",
                    "task_description": "これまでの分析結果を総合的に解釈し、ビジネス上の示唆をまとめてください。"
                },
                "id": "call_interpret_1"
            }]
        ),
        # interpret_nodeへの具体的な指示メッセージ
        ToolMessage(
            content="月別売上の合計が知りたいです。",
            tool_call_id="call_interpret_1"
        )
    ],
    
    # df_historyに、過去のノードが生成したデータを設定
    df_history=[
        {"月別売上の取得": sales_data},
        {"カテゴリ別顧客数の取得": customer_data}
    ]
)
res = processing_node(test_state_for_process)




> Entering new AgentExecutor chain...
```python
print(df0)
result = df0.groupby('月')['売上'].sum().reset_index()
print(result)
```

> Finished chain.
データ加工中にエラーが発生しました: エージェントは加工後のDataFrameを返しませんでした。


In [9]:
import pandas as pd
import plotly.express as px
import streamlit as st
from typing import Any, List
import os

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# --- データ準備 ---
data = {
    'date': ['2025-06-01', '2025-06-01', '2025-06-01', '2025-06-02', '2025-06-02', '2025-06-02', '2025-06-03', '2025-06-03', '2025-06-03', '2025-06-04', '2025-06-04', '2025-06-04'],
    'product': ['Apple', 'Orange', 'Banana', 'Apple', 'Orange', 'Grape', 'Apple', 'Banana', 'Orange', 'Grape', 'Apple', 'Banana'],
    'sales': [150, 80, 120, 180, 95, 200, 210, 140, 85, 220, 190, 130]
}
df = pd.DataFrame(data)

# Step 1: コールバックを定義
class CodeCollectorCallback(BaseCallbackHandler):
    def __init__(self):
        self.codes = []
    
    # ★★★ on_agent_action から on_tool_start に変更 ★★★
    def on_tool_start(self, serialized: dict, input_str: str, **kwargs: Any) -> Any:
        """ツールが実行を開始する直前に呼び出される"""
        # 実行されるツールの名前が "python_repl" かどうかをチェック
        if serialized.get("name") == "python_repl":
            # ツールの入力（Pythonコード）をリストに追加
            self.codes.append(input_str)
import json
from langchain.callbacks.base import BaseCallbackHandler
from typing import Any, Dict, List
from langchain.schema import LLMResult, AgentFinish
class LogCollectorCallback(BaseCallbackHandler):
    # (上記のクラス定義をここに貼り付け)
    def __init__(self):
        super().__init__()
        self.logs = []
    def _format_log(self, event_name: str, data: Any) -> str:
        if isinstance(data, dict): data_str = json.dumps(data, indent=2, ensure_ascii=False)
        else: data_str = str(data)
        return f"--- {event_name} ---\n{data_str}\n"
    def on_chain_start(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any) -> Any: self.logs.append(self._format_log("⛓️ Chain Start", {"inputs": inputs}))
    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> Any: self.logs.append(self._format_log("🧠 LLM Start", {"prompts": prompts}))
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any: self.logs.append(self._format_log("✅ LLM End", {"response": response.generations[0][0].text}))
    def on_tool_start(self, serialized: Dict[str, Any], input_str: str, **kwargs: Any) -> Any: self.logs.append(self._format_log(f"🛠️ Tool Start: {serialized.get('name')}", {"input": input_str}))
    def on_tool_end(self, output: str, **kwargs: Any) -> Any: self.logs.append(self._format_log("✅ Tool End", {"output": output}))
    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any: self.logs.append(self._format_log("🏁 Agent Finish", {"return_values": finish.return_values}))
    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> Any: self.logs.append(self._format_log("🎉 Chain End", {"outputs": outputs}))


# Step 2: エージェントを準備
# ★★★ 修正点1: ツールに明示的な名前を付け、descriptionをシンプルに
python_tool = PythonAstREPLTool(
    name="python_repl", 
    locals={"df": df, "px": px, "pd": pd}, 
    description="Pythonコードを実行して、渡されたdfを分析・可視化します。"
)

# LLMの準備
llm_model_name = os.getenv("LLM_MODEL_NAME", "gemini-1.5-flash")
google_api_key = os.getenv("GOOGLE_API_KEY") 
llm = ChatGoogleGenerativeAI(model=llm_model_name, temperature=0, google_api_key=google_api_key)

# プロンプトテンプレートの準備
system_prompt = """
あなたはデータ分析のエキスパートです。ユーザーの指示に従い、ツールを使って分析し、結果を返してください。

- 渡された`df`というDataFrameを操作・分析してください。
- 可視化を行う場合、最終的なグラフオブジェクトは必ず `fig` という変数に格納してください。
- データフレームを最終成果物とする場合、そのデータフレームは `final_df` という変数に格納してください。

思考の過程:
{agent_scratchpad}
"""

tools = [python_tool]
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# エージェントの組み立て
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# --- ここからがメインの実行フロー ---

# ★★★ 修正点2: ユーザー指示を実態に合わせる
user_instruction = "渡されたDataFrame `df` を使って、日別の売上(sales)合計を計算し、その結果を棒グラフで表示してください。"


# コールバックのインスタンスを作成
code_collector = CodeCollectorCallback()
# verbose_logger = LogCollectorCallback()

# エージェントを実行し、両方のコールバックを渡す
result = agent_executor.invoke(
    {"input": user_instruction},
    {"callbacks": [code_collector, verbose_logger]} # リストで複数渡せる
)
result = agent_executor.invoke(
    {"input": user_instruction},
    {"callbacks": [code_collector]} # リストで複数渡せる
)



> Entering new AgentExecutor chain...


> Finished chain.


In [10]:
print(result)

{'input': '渡されたDataFrame `df` を使って、日別の売上(sales)合計を計算し、その結果を棒グラフで表示してください。', 'output': ''}


In [10]:

collected_codes = code_collector.codes
collected_codes

[]

In [ ]:

collected_codes = code_collector.codes
st.code("\n\n".join(collected_codes), language="python")

# Step 4: メインプログラム側で、収集したコードを逐次実行
st.subheader("収集したコードの実行と結果の可視化")

# 実行環境のスコープを準備
# このスコープを使い回すことで、前のコードの変数を次のコードで使える
execution_scope = {
    "pd": pd,
    "px": px
}

if collected_codes:
    try:
        # 収集したコードを順番に実行
        for code in collected_codes:
            exec(code, execution_scope)
        
        # Step 5: 最終的なオブジェクトをスコープから取得して利用
        fig = execution_scope.get('fig')
        final_df = execution_scope.get('final_df')

        if fig:
            st.success("グラフオブジェクト(fig)を取得しました！")
            st.plotly_chart(fig)
        elif final_df is not None:
            st.success("DataFrame(final_df)を取得しました！")
            st.dataframe(final_df)
        else:
            st.warning("最終成果物 (fig または final_df) が見つかりませんでした。")

    except Exception as e:
        st.error(f"コードの実行中にエラーが発生しました: {e}")
else:
    st.warning("実行可能なコードが生成されませんでした。")